In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras.models import load_model
import numpy as np
import pickle
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import pickle
import os
import re

In [15]:
#downloded the 2 datasets from https://www.gutenberg.org/
#Reading dataset1 as text file
dataset1= open("data/dataset1.txt", "r", encoding = "utf8")

# store file in list
data_lines_1 = []
for lines in dataset1:
    data_lines_1.append(lines)

# Convert list to string
data1 = ""
for words in data_lines_1:
    data1= ' '. join(data_lines_1) 

In [16]:
#Reading dataset2 as text file
dataset2= open("data/dataset2.txt", "r", encoding = "utf8")

# store file in list
data_lines_2 = []
for lines in dataset2:
    data_lines_2.append(lines)

# Convert list to string
data2 = ""
for words in data_lines_2:
    data2= ' '. join(data_lines_2) 

In [17]:
#concatinating the 2 strings from two different datasets 
data=data1 + ' ' + data2

In [18]:
data[1:100]

"\n Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle\n \n This eBook is for"

In [19]:
def cleaning(text):
    global data
    data = text.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','') 
    #new line, carriage return, unicode character --> replace by space
    data=data.replace(',','').replace(":","")
    data = re.sub(r"[^a-zA-Z0-9 ]", "", data)   
    data = data.split()
    data = ' '.join(data)
    return data
cleaning(data)

'Project Gutenbergs The Adventures of Sherlock Holmes by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever You may copy it give it away or reuse it under the terms of the Project Gutenberg License included with this eBook or online at wwwgutenbergnet Title The Adventures of Sherlock Holmes Author Arthur Conan Doyle Release Date November 29 2002 EBook 1661 Last Updated May 20 2019 Language English Character set encoding UTF8 START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES Produced by an anonymous Project Gutenberg volunteer and Jose Menendez cover The Adventures of Sherlock Holmes by Arthur Conan Doyle Contents I A Scandal in Bohemia II The RedHeaded League III A Case of Identity IV The Boscombe Valley Mystery V The Five Orange Pips VI The Man with the Twisted Lip VII The Adventure of the Blue Carbuncle VIII The Adventure of the Speckled Band IX The Adventure of the Engineers Thumb X The Adventur

In [13]:
print("Length of the corpus:",len(data))

Length of the corpus: 997593


In [6]:
#Tokenizing the data or splitting into chunks
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[136, 4601, 1, 952, 4, 123, 33, 45, 523, 2104, 2105, 27, 953, 14, 22]

In [7]:
#Finding the sequential data and its length
len(sequence_data)

107547

In [8]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)


8722


In [9]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  107544


array([[ 136, 4601,    1,  952],
       [4601,    1,  952,    4],
       [   1,  952,    4,  123],
       [ 952,    4,  123,   33],
       [   4,  123,   33,   45],
       [ 123,   33,   45,  523],
       [  33,   45,  523, 2104],
       [  45,  523, 2104, 2105],
       [ 523, 2104, 2105,   27],
       [2104, 2105,   27,  953]])

In [10]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [11]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[ 136 4601    1]
 [4601    1  952]
 [   1  952    4]
 [ 952    4  123]
 [   4  123   33]
 [ 123   33   45]
 [  33   45  523]
 [  45  523 2104]
 [ 523 2104 2105]
 [2104 2105   27]]
Response:  [ 952    4  123   33   45  523 2104 2105   27  953]


In [12]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

2022-11-02 12:42:18.945852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             87220     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8722)              8730722   
                                                                 
Total params: 21,866,942
Trainable params: 21,866,942
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

#epoch needs to be increased but right now limited to 30 due to system specification
#Will increase it to 100 post aws deployment

history=model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint]) 

Epoch 1/30
1681/1681 [==============================] - ETA: 0s - loss: 6.3662
Epoch 1: loss improved from inf to 6.36619, saving model to next_words.h5
1681/1681 [==============================] - 246s 145ms/step - loss: 6.3662
Epoch 2/30
1681/1681 [==============================] - ETA: 0s - loss: 5.7685
Epoch 2: loss improved from 6.36619 to 5.76846, saving model to next_words.h5
1681/1681 [==============================] - 241s 143ms/step - loss: 5.7685
Epoch 3/30
1681/1681 [==============================] - ETA: 0s - loss: 5.4492
Epoch 3: loss improved from 5.76846 to 5.44920, saving model to next_words.h5
1681/1681 [==============================] - 240s 143ms/step - loss: 5.4492
Epoch 4/30
1681/1681 [==============================] - ETA: 0s - loss: 5.1968
Epoch 4: loss improved from 5.44920 to 5.19681, saving model to next_words.h5
1681/1681 [==============================] - 239s 142ms/step - loss: 5.1968
Epoch 5/30
1681/1681 [==============================] - ETA: 0s - loss: 

In [ ]:
loss=history.history["loss"]
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label="Training loss")
plt.title("Training loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

In [3]:
# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def next_word_predictor(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""
  
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
  
    print(predicted_word)
    return predicted_word

2022-11-02 15:16:08.313317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
while(True):
    text = input("Enter your line: ")
  
    if text == "0":
        print("Execution completed.....")
        break
  
    else:
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)
        
            next_word_predictor(model, tokenizer, text)
          
        except Exception as e:
            print("Error occurred: ",e)
            continue

Enter your line: how are you
['how', 'are', 'you']
1/1 [==============================] - 0s 406ms/step
getting
Enter your line: distracting factor which
['distracting', 'factor', 'which']
1/1 [==============================] - 0s 25ms/step
might
Enter your line: I had better
['I', 'had', 'better']
1/1 [==============================] - 0s 25ms/step
postpone
Enter your line: Can you please
['Can', 'you', 'please']
1/1 [==============================] - 0s 24ms/step
herself
Enter your line: what is going
['what', 'is', 'going']
1/1 [==============================] - 0s 73ms/step
on
Enter your line: this is my
['this', 'is', 'my']
1/1 [==============================] - 0s 24ms/step
friend
